In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
df = pd.read_csv(r'C:\Users\MedTech\Desktop\02.12.25\test_patients_bsk_20251202.csv')

In [ ]:
def convert_date_format(df, date_columns):
    """
    Преобразует даты из формата DD.MM.YYYY в YYYY-MM-DD
    """
    for col in date_columns:
        if col in df.columns:
            # Преобразуем даты
            df[col] = pd.to_datetime(df[col], format='%d.%m.%Y', errors='coerce')
            # Конвертируем обратно в строку в нужном формате
            df[col] = df[col].dt.strftime('%Y-%m-%d')
            # Для None/NaT значений оставляем пустую строку
            df[col] = df[col].replace('NaT', '').replace('nan', '')
    
    return df

def clean_and_prepare_csv(input_file, output_file, date_columns=None):
    """
    Читает CSV, преобразует даты и сохраняет в новый CSV
    """
    # Если не указаны колонки с датами, определим автоматически
    if date_columns is None:
        date_columns = []
    
    # Читаем CSV файл
    # Параметр dtype=str гарантирует, что все данные будут как строки
    df = pd.read_csv(input_file, dtype=str, na_filter=False)
    
    # Автоматическое определение колонок с датами (по названию)
    auto_date_cols = [col for col in df.columns if 'date' in col.lower()]
    
    # Объединяем указанные и автоматически найденные колонки
    all_date_columns = list(set(date_columns + auto_date_cols))
    
    print(f"Найдены колонки: {list(df.columns)}")
    print(f"Колонки для преобразования дат: {all_date_columns}")
    
    # Преобразуем даты
    df = convert_date_format(df, all_date_columns)
    
    # Проверяем результат
    print("\nПример преобразованных данных:")
    print(df.head())
    
    # Проверяем некорректные даты
    for col in all_date_columns:
        if col in df.columns:
            invalid_dates = df[df[col] == '']
            if len(invalid_dates) > 0:
                print(f"\nВнимание! В колонке '{col}' найдены некорректные даты: {len(invalid_dates)} шт.")
    
    # Сохраняем результат
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\nФайл сохранен: {output_file}")
    
    return df